In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [4]:
# بارگذاری داده از UCI
df = fetch_openml(name='Banknote-authentication', version=1, as_frame=True)
data = df['frame'][['V1', 'V2', 'Class']]  # فقط دو ویژگی و برچسب

# جدا کردن ویژگی‌ها و برچسب
X = data[['V1', 'V2']].values
y = data['Class'].astype(int).values

# تقسیم داده به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [5]:
# تابع احتمال چگالی گوسی (برای ویژگی‌های پیوسته)
def gaussian_pdf(x, mean, var):
    eps = 1e-6  # جلوگیری از تقسیم بر صفر
    coef = 1.0 / np.sqrt(2.0 * np.pi * var + eps)
    exponent = np.exp(-((x - mean) ** 2) / (2 * var + eps))
    return coef * exponent

# آموزش مدل: محاسبه میانگین، واریانس، و احتمال پیشین
def train_naive_bayes(X, y):
    summaries = {}
    classes = np.unique(y)
    for c in classes:
        X_c = X[y == c]
        means = X_c.mean(axis=0)
        variances = X_c.var(axis=0)
        prior = X_c.shape[0] / X.shape[0]
        summaries[c] = {'mean': means, 'var': variances, 'prior': prior}
    return summaries

# پیش‌بینی یک نمونه
def predict_single(x, summaries):
    posteriors = {}
    for c, stats in summaries.items():
        prior = np.log(stats['prior'])  # استفاده از لگاریتم برای پایداری عددی
        likelihoods = np.sum(np.log(gaussian_pdf(x, stats['mean'], stats['var'])))
        posteriors[c] = prior + likelihoods
    return max(posteriors, key=posteriors.get)

# پیش‌بینی کل مجموعه
def predict(X, summaries):
    return np.array([predict_single(x, summaries) for x in X])


In [6]:
# آموزش مدل
summaries = train_naive_bayes(X_train, y_train)

# پیش‌بینی
y_pred = predict(X_test, summaries)

# ارزیابی
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8592233009708737

Classification Report:
               precision    recall  f1-score   support

           1       0.85      0.91      0.88       229
           2       0.88      0.79      0.83       183

    accuracy                           0.86       412
   macro avg       0.86      0.85      0.86       412
weighted avg       0.86      0.86      0.86       412

